In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/cambrian_reco/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/cambrian_reco/')
import pandas as pd

os.environ['PATH'] = '/Users/username/anaconda3/envs/recoenv/bin:/Users/username/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
from src.ArticleSelectionService import ArticleSelectionService

In [3]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [7]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT DISTINCT parent_id, parent_name FROM cluster_hierarchy WHERE child_left_name = 'category'"
    cursor.execute(sql)
    results = cursor.fetchall()

In [9]:
categories = pd.DataFrame(results, columns = ['cluster_id', 'cluster_name'])

In [10]:
categories

,cluster_id,cluster_name
0,147,Real Estate and Infrastructure
1,139,Commodities and Energy Sector
2,141,Economic Updates and Policies
3,144,"International Markets, Business and Trade"
4,145,Non financial or business
5,143,Indian Stock Markets and Trading
6,140,Corporate News and Mergers
7,142,Financial reports and earning calls
8,148,Technology and Startups
9,146,Personal Finance and Investments


In [29]:
user_pref = []

In [30]:
for user_id in range(100,174):
    cur_user_pref = ArticleSelectionService.compute_cluster_preferences(user_id,[])
    cur_user_pref['user_id'] = user_id
    user_pref.append(cur_user_pref)

In [31]:
pref_df = pd.concat(user_pref)

In [32]:
pref_df = pd.merge(pref_df, categories, how = 'left', on = 'cluster_id')

In [33]:
pref_df['implicit_a'].mean()

0.15251802501259024

In [34]:
pref_df['implicit_b'].mean()

2.957426221252308

In [35]:
cum = pref_df.groupby('cluster_name')[['implicit_a', 'implicit_b']].sum().reset_index()

In [36]:
cum['ratio'] = cum['implicit_a']/cum['implicit_b']

In [65]:
pref_df['prob'] = beta(pref_df['cluster_a'], pref_df['cluster_b'] )

In [67]:
pref_df.sample(20)

,cluster_id,explicit_a,explicit_b,implicit_a,implicit_b,cluster_a,cluster_b,user_id,cluster_name,prob
435,148,1.0,6.0,1.000000,1.827826,2.000000,7.827826,143,Technology and Startups,0.261989
673,139,1.0,6.0,0.000000,2.905714,1.000000,8.905714,167,Commodities and Energy Sector,0.078245
347,146,1.0,6.0,0.000000,0.000000,1.000000,6.000000,134,Personal Finance and Investments,0.250285
143,139,1.0,6.0,0.000000,0.000000,1.000000,6.000000,114,Commodities and Energy Sector,0.081494
510,142,1.0,6.0,0.000000,0.000000,1.000000,6.000000,151,Financial reports and earning calls,0.099283
167,146,1.0,6.0,0.000000,0.000000,1.000000,6.000000,116,Personal Finance and Investments,0.276633
538,143,1.0,6.0,0.000000,4.569565,1.000000,10.569565,153,Indian Stock Markets and Trading,0.075160
539,147,1.0,6.0,0.000000,0.000000,1.000000,6.000000,153,Real Estate and Infrastructure,0.119993
311,141,1.0,6.0,0.000000,12.000000,1.000000,18.000000,131,Economic Updates and Policies,0.039885
140,142,1.0,6.0,0.000000,0.000000,1.000000,6.000000,114,Financial reports and earning calls,0.113835


In [5]:
k = ArticleSelectionService.get_feed(user_id=120,cur_feed_article_ids=[],feed_article_count=10,session_page=0,topic_ids=[],seconds_since_last_api_call=1000,content_type='ARTICLE')

completed feed computation in 962.028980255127  ms


In [5]:
from numpy.random import beta

In [6]:
k

{'feedContents': [{'content_id': '65e1b5257825a96cf7dc402b'},
  {'content_id': '65e1d1537825a96cf7dc4182'},
  {'content_id': '65e0b8077825a96cf7dc2feb'},
  {'content_id': '65e160cc7825a96cf7dc3df2'},
  {'content_id': '65e0d42e7825a96cf7dc33bf'},
  {'content_id': '65df668fb25990560f7e0403'},
  {'content_id': '65e1b5257825a96cf7dc402a'},
  {'content_id': '65e1d1557825a96cf7dc4194'},
  {'content_id': '65e1b52e7825a96cf7dc404b'},
  {'content_id': '65e1990a7825a96cf7dc3f65'}],
 'model': 'thompson_sampling_llm_v3',
 'additionalInfo': {'65e1b5257825a96cf7dc402b': {'source_id': '65768d2e7c73e77fdb13a9b3',
   'hours_since_publication': 10.684175747777777,
   'cluster_id': 148,
   'article_a': 8.5,
   'article_b': 13.0,
   'prior_a': 8.0,
   'prior_b': 10.0,
   'cluster_name': 'Technology and Startups',
   'explicit_a': 1.0,
   'explicit_b': 15.0,
   'implicit_a': 0.0,
   'implicit_b': 24.205217391304345,
   'cluster_a': 1.0,
   'cluster_b': 39.205217391304345,
   'timeliness_score': 0.872178926

In [37]:
cum

,cluster_name,implicit_a,implicit_b,ratio
0,Commodities and Energy Sector,2.585000,108.532236,0.023818
1,Corporate News and Mergers,16.794224,404.179441,0.041551
2,Economic Updates and Policies,20.172702,344.418634,0.058570
3,Financial reports and earning calls,0.000000,34.129503,0.000000
4,Indian Stock Markets and Trading,42.271413,600.893385,0.070348
5,"International Markets, Business and Trade",0.913913,122.011646,0.007490
6,Non financial or business,3.290000,103.633416,0.031747
7,Personal Finance and Investments,16.490000,226.165031,0.072911
8,Real Estate and Infrastructure,1.803913,25.717081,0.070145
9,Technology and Startups,8.542174,218.815031,0.039038


In [3]:
from src._utils import get_popularity_priors_for_candidates

In [4]:
popularity_df = get_popularity_priors_for_candidates('ARTICLE')

In [8]:
popularity_df.dtypes

article_id                         object
source_id                          object
cluster_id                        float64
published_time             datetime64[ns]
activity_date              datetime64[ns]
feed_a                              int64
feed_summary_a                      int64
total_feed_a                      float64
feed_b                              int64
search_a                            int64
search_summary_a                  float64
total_search_a                     object
search_b                            int64
prior_a                             int64
prior_b                             int64
days_since_activity               float64
activity_decay                    float64
hours_since_publication           float64
dtype: object

In [7]:
popularity_df['total_feed_a'] = popularity_df['total_feed_a'].astype(float)

In [6]:
popularity_df = popularity_df.sort_values('prior_a',ascending=False)

In [9]:
import numpy as np

In [23]:
%timeit -n 7 np.mean([popularity_df['prior_a'], popularity_df['prior_b']],axis=0)

The slowest run took 11.25 times longer than the fastest. This could mean that an intermediate result is being cached.
101 µs ± 120 µs per loop (mean ± std. dev. of 7 runs, 7 loops each)


In [14]:
%timeit -n 7 np.mean(popularity_df['article_a'] + popularity_df['article_b'],axis=0)

232 µs ± 91 µs per loop (mean ± std. dev. of 7 runs, 7 loops each)


In [12]:
%timeit -n 7 (popularity_df['article_a'] + popularity_df['article_b'])/3

222 µs ± 115 µs per loop (mean ± std. dev. of 7 runs, 7 loops each)


In [7]:
popularity_df

,article_id,source_id,hours_since_publication,cluster_id,article_a,article_b,prior_a,prior_b
1540,65db01991f6d9144de7edffa,6555987996fb32eef21a562b,177.331010,148,11.0,10.0,11.0,10.0
386,65d2fc6c504cea1b1d2eecbb,65768d2e7c73e77fdb13a9b3,275.552122,148,11.0,10.0,11.0,10.0
348,65d2c431abb31841109d0f12,6560b3a49952a16bbc397225,279.231844,139,11.0,10.0,11.0,10.0
701,65d59f79504cea1b1d2f0dba,65768d2e7c73e77fdb13a9b3,231.219899,148,11.0,10.0,11.0,10.0
477,65d36cf7504cea1b1d2ef092,6555987996fb32eef21a562b,313.842955,141,11.0,10.0,11.0,10.0
...,...,...,...,...,...,...,...,...
2660,65e02b797825a96cf7dc24bd,6555987996fb32eef21a562b,82.630177,147,1.0,10.0,1.0,10.0
2659,65e02b797825a96cf7dc24bb,6555987996fb32eef21a562b,81.965177,148,1.0,10.0,1.0,10.0
2658,65e02b797825a96cf7dc24ba,6555987996fb32eef21a562b,83.231844,140,1.0,10.0,1.0,10.0
2657,65e02b797825a96cf7dc24b9,6555987996fb32eef21a562b,82.200177,141,1.0,10.0,1.0,10.0


In [41]:
popularity_df['interacted_a'] = popularity_df['article_a'] - popularity_df['prior_a']
popularity_df['interacted_b'] = popularity_df['article_b'] - popularity_df['prior_b']

In [46]:
popularity_df = pd.merge(popularity_df, categories, how = 'left', on = 'cluster_id')

In [47]:
popularity_df['interacted_a'].sum()

130.48583060118293

In [48]:
popularity_df['interacted_b'].sum()

3848.100739016337

In [52]:
pop_cum = popularity_df.groupby('cluster_name')[['interacted_a','interacted_b']].sum().reset_index()

In [53]:
pop_cum['ratio'] = pop_cum['interacted_a']/pop_cum['interacted_b']

In [54]:
pop_cum

,cluster_name,interacted_a,interacted_b,ratio
0,Commodities and Energy Sector,5.001672,261.409551,0.019133
1,Corporate News and Mergers,26.257275,885.334277,0.029658
2,Economic Updates and Policies,22.411064,627.524243,0.035713
3,Financial reports and earning calls,1.235275,46.048303,0.026826
4,Indian Stock Markets and Trading,27.684002,761.082497,0.036375
5,"International Markets, Business and Trade",1.377391,93.864393,0.014674
6,Non financial or business,6.086320,111.018126,0.054823
7,Personal Finance and Investments,18.874614,386.927909,0.048781
8,Real Estate and Infrastructure,7.481758,147.650601,0.050672
9,Technology and Startups,14.076460,527.240840,0.026698


In [61]:
popularity_df

,article_id,source_id,hours_since_publication,cluster_id,article_a,article_b,prior_a,prior_b,interacted_a,interacted_b,cluster_name
0,652ebb751e5cc42b1b1398c3,65291b1e9a2fbc229e5f29c9,3583.344092,143,8.0,10.00,8.0,10.0,0.0,0.00,Indian Stock Markets and Trading
1,652ebb761e5cc42b1b1398c4,65291b1e9a2fbc229e5f29c9,3556.015203,143,7.0,10.00,7.0,10.0,0.0,0.00,Indian Stock Markets and Trading
2,652ebb781e5cc42b1b1398c5,65291b1e9a2fbc229e5f29c9,3385.312148,146,8.0,10.94,8.0,10.0,0.0,0.94,Personal Finance and Investments
3,652ebb7e1e5cc42b1b1398cd,65291b1e9a2fbc229e5f29c9,3509.503537,141,6.0,10.00,6.0,10.0,0.0,0.00,Economic Updates and Policies
4,652ebb861e5cc42b1b1398e8,65291b1e9a2fbc229e5f29c9,3409.964648,143,8.0,10.00,8.0,10.0,0.0,0.00,Indian Stock Markets and Trading
...,...,...,...,...,...,...,...,...,...,...,...
3501,65e17ced7825a96cf7dc3ebb,650046bd005149c49201269f,2.072703,141,7.0,11.00,7.0,10.0,0.0,1.00,Economic Updates and Policies
3502,65e17d137825a96cf7dc3ed5,6560ac8b9952a16bbc397223,7.852981,140,1.0,10.00,1.0,10.0,0.0,0.00,Corporate News and Mergers
3503,65e17d197825a96cf7dc3f07,6560ac8b9952a16bbc397223,8.961315,143,1.0,10.00,1.0,10.0,0.0,0.00,Indian Stock Markets and Trading
3504,65e17d1b7825a96cf7dc3f24,6560ac8b9952a16bbc397223,8.708537,139,1.0,10.00,1.0,10.0,0.0,0.00,Commodities and Energy Sector


In [63]:
from numpy.random import beta

In [62]:
popularity_df[(popularity_df['interacted_a'] + popularity_df['interacted_b']) > 0][['prior_a', 'interacted_a']].corr()

,prior_a,interacted_a
prior_a,1.000000,0.122003
interacted_a,0.122003,1.000000


In [58]:
popularity_df.nlargest(20,'interacted_b')

,article_id,source_id,hours_since_publication,cluster_id,article_a,article_b,prior_a,prior_b,interacted_a,interacted_b,cluster_name
1635,65dbfeb31f6d9144de7ee399,6560af039952a16bbc397224,107.005481,143,7.222500,33.594348,7.0,10.0,0.222500,23.594348,Indian Stock Markets and Trading
1634,65dbfeb31f6d9144de7ee398,6560b3a49952a16bbc397225,108.352981,141,14.676087,29.024783,9.0,10.0,5.676087,19.024783,Economic Updates and Policies
1639,65dbfeb41f6d9144de7ee39f,6560af039952a16bbc397224,107.086592,141,9.913913,28.063043,9.0,10.0,0.913913,18.063043,Economic Updates and Policies
1636,65dbfeb41f6d9144de7ee39b,6560af039952a16bbc397224,107.727703,141,9.136413,27.173043,8.0,10.0,1.136413,17.173043,Economic Updates and Policies
1357,65d993e51f6d9144de7edac9,650046bd005149c49201269f,145.895481,146,8.000000,27.048033,8.0,10.0,0.000000,17.048033,Personal Finance and Investments
1631,65dbfeae1f6d9144de7ee36b,6560af039952a16bbc397224,108.574926,141,7.000000,26.283043,7.0,10.0,0.000000,16.283043,Economic Updates and Policies
1403,65d9cc241f6d9144de7edb8b,6560af039952a16bbc397224,148.296037,141,8.913913,25.334254,8.0,10.0,0.913913,15.334254,Economic Updates and Policies
1330,65d95bab1f6d9144de7eda21,6560af039952a16bbc397224,155.983259,143,8.211923,23.372074,8.0,10.0,0.211923,13.372074,Indian Stock Markets and Trading
1089,65d80a31240b44151528cae1,6560af039952a16bbc397224,180.378537,146,8.566614,23.357971,7.0,10.0,1.566614,13.357971,Personal Finance and Investments
664,65d56731504cea1b1d2f0bcd,6560af039952a16bbc397224,228.141870,146,10.370870,23.002508,9.0,10.0,1.370870,13.002508,Personal Finance and Investments


In [45]:
130/3900

0.03333333333333333